In [1]:
import pprint

from qiskit.providers.fake_provider import FakeManila

from casq.casq_simulator import CasqSimulator
from casq.gates.drag_pulse_gate import DragPulseGate
from casq.gates.gaussian_pulse_gate import GaussianPulseGate


In [19]:
qubit = 0
duration = 256
amplitude = 1
sigma = 128
beta = 0.75
pulse0 = GaussianPulseGate(duration, amplitude, sigma)
circuit0 = pulse0.circuit(qubit)
pulse1 = DragPulseGate(duration, amplitude, sigma, beta)
circuit1 = pulse1.circuit(qubit)


In [20]:
simulator = CasqSimulator(FakeManila(), [0])


In [21]:
result = simulator.run(circuit0).results[0]
print(result.data.counts)


{'0': 44, '1': 980}


In [22]:
result = simulator.run(circuit1).results[0]
print(result.data.counts)


{'0': 47, '1': 977}
